## Sixth Session (Related to the Course Project)

---------------

## Graph Classification with [Deep Graph Library (DGL)](https://docs.dgl.ai/index.html) for the graduate course "[Graph Machine learning](https://github.com/zahta/graph_ml)"

### Dataset: bbbp

##### by [Zahra Taheri](https://github.com/zahta), 06 June 2023

---------------

### This Tutorial Is Prepared Based on the Following References

- [FunQG: Molecular Representation Learning via Quotient Graphs](https://pubs.acs.org/doi/10.1021/acs.jcim.3c00445)
- [Supporting Information of FunQG](https://pubs.acs.org/doi/suppl/10.1021/acs.jcim.3c00445/suppl_file/ci3c00445_si_001.pdf)
- [GitHub Repository of FunQG](https://github.com/hhaji/funqg)

In [3]:
pip install  dgl -f https://data.dgl.ai/wheels/repo.html

Looking in links: https://data.dgl.ai/wheels/repo.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 46.1 MB/s eta 0:00:00


In [ ]:
#pip install  dgl -f https://data.dgl.ai/wheels/cu113/repo.html

In [ ]:
#pip install  dglgo -f https://data.dgl.ai/wheels-test/repo.html

In [2]:
pip install  dglgo -f https://data.dgl.ai/wheels-test/repo.html

Looking in links: https://data.dgl.ai/wheels-test/repo.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.5/63.5 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.4/52.4 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 55.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 56.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.7/572.7 kB 57.2 MB/s eta 0:00:00
  Created whee

In [5]:
%matplotlib inline
import os

os.environ["DGLBACKEND"] = "pytorch"
import dgl
import numpy as np
import networkx as nx
import torch
import torch.nn as nn
import dgl.function as fn
import torch.nn.functional as F
import shutil
from torch.utils.data import DataLoader
import cloudpickle
from dgl.nn import GraphConv
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from dgl.nn import GraphConv
from dgl.nn import GINConv
from dgl.nn import SAGEConv
from dgl.nn import GATConv

#### Set Path

In [6]:
!pip install unzip

  Preparing metadata (setup.py) ... done
  Created wheel for unzip: filename=unzip-1.0.0-py3-none-any.whl size=1279 sha256=8d395ed526a9d452eae06645515c273c9a8581b8f874fb62b2ccc2b2c9a9b111
  Stored in directory: /root/.cache/pip/wheels/80/dc/7a/f8af45bc239e7933509183f038ea8d46f3610aab82b35369f4
Successfully built unzip


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
!unzip /content/drive/MyDrive/graph_data1.zip

Archive:  /content/drive/MyDrive/graph_data1.zip
  inflating: scaffold_0_smiles_train.pickle  
  inflating: scaffold_0_test.bin     
  inflating: scaffold_0_val.bin      
  inflating: scaffold_0_smiles_val.pickle  
  inflating: scaffold_0_smiles_test.pickle  
  inflating: scaffold_0_train.bin    


In [9]:
# Define the path to the current directory where the ZIP file is located.
current_dir = "/content/drive/MyDrive/graph_data1.zip"

# Create the path to the directory where model checkpoints will be saved.
checkpoint_path = current_dir + "save_models/model_checkpoints/" + "checkpoint"
os.makedirs(checkpoint_path, exist_ok=True)

# Define the path to the directory where the best model will be saved.
best_model_path = current_dir + "save_models/best_model/"

# Create a temporary folder path for data manipulation.
folder_data_temp = current_dir + "data_temp/"

# Remove the temporary folder if it exists, ignoring any errors if it does not.
shutil.rmtree(folder_data_temp, ignore_errors=True)

# Define the path to save the unpacked files from the ZIP archive.
path_save = current_dir

# Unpack the contents of the ZIP archive to the temporary folder.
shutil.unpack_archive(path_save, folder_data_temp)


#### Custom PyTorch Datasets

In [10]:
""" Classification Dataset """

class DGLDatasetClass(torch.utils.data.Dataset):
    def __init__(self, address):
        # Initialize the dataset with the given address
        self.address = address + ".bin"
        self.list_graphs, train_labels_masks_globals = dgl.load_graphs(self.address)
        num_graphs = len(self.list_graphs)

        # Extract labels, masks, and globals from train_labels_masks_globals
        self.labels = train_labels_masks_globals["labels"].view(num_graphs, -1)
        self.masks = train_labels_masks_globals["masks"].view(num_graphs, -1)
        self.globals = train_labels_masks_globals["globals"].view(num_graphs, -1)

    def __len__(self):
        # Return the length of the dataset
        return len(self.list_graphs)

    def __getitem__(self, idx):
        # Retrieve the item at the given index
        return self.list_graphs[idx], self.labels[idx], self.masks[idx], self.globals[idx]


#### Defining Train, Validation, and Test Set

In [11]:
# Define the path for temporary data
path_data_temp = folder_data_temp + "scaffold" + "_" + str(0)

# Create instances of DGLDatasetClass for the train, validation, and test sets
train_set = DGLDatasetClass(address=path_data_temp + "_train")
val_set = DGLDatasetClass(address=path_data_temp + "_val")
test_set = DGLDatasetClass(address=path_data_temp + "_test")

# Print the lengths of the train, validation, and test sets
print(len(train_set), len(val_set), len(test_set))

1631 203 205


#### Data Loader

In [12]:
# Define a collate function to process a batch of data samples.
def collate(batch):
    # Extract the graphs from the batch and create a batched graph using dgl.batch.
    graphs = [e[0] for e in batch]
    g = dgl.batch(graphs)

    # Extract the labels from the batch and stack them into a tensor.
    labels = [e[1] for e in batch]
    labels = torch.stack(labels, 0)

    # Extract the masks from the batch and stack them into a tensor.
    masks = [e[2] for e in batch]
    masks = torch.stack(masks, 0)

    # Extract the global features from the batch and stack them into a tensor.
    globals = [e[3] for e in batch]
    globals = torch.stack(globals, 0)

    # Return the batched graph, labels, masks, and globals.
    return g, labels, masks, globals


# Define a loader function to create data loaders for the training, validation, and test sets.
def loader(batch_size=64):
    # Create a data loader for the training set.
    train_dataloader = DataLoader(train_set,
                                  batch_size=batch_size,
                                  collate_fn=collate,
                                  drop_last=False,
                                  shuffle=True,
                                  num_workers=1)

    # Create a data loader for the validation set.
    val_dataloader = DataLoader(val_set,
                                batch_size=batch_size,
                                collate_fn=collate,
                                drop_last=False,
                                shuffle=False,
                                num_workers=1)

    # Create a data loader for the test set.
    test_dataloader = DataLoader(test_set,
                                 batch_size=batch_size,
                                 collate_fn=collate,
                                 drop_last=False,
                                 shuffle=False,
                                 num_workers=1)

    # Return the data loaders for training, validation, and test sets.
    return train_dataloader, val_dataloader, test_dataloader


In [13]:
# Create data loaders for the training, validation, and test sets with a batch size of 64.
train_dataloader, val_dataloader, test_dataloader = loader(batch_size=64)

#### Defining A GNN

##### Some Variables

In [14]:
#Bace dataset has 1 task. Some other datasets may have some more number of tasks, e.g., tox21 has 12 tasks.
num_tasks = 1

# Size of global feature of each graph
global_size = 200

# Number of epochs to train the model
num_epochs = 100

# Number of steps to wait if the model performance on the validation set does not improve
patience = 10

#Configurations to instantiate the model
config = {"node_feature_size":127, "edge_feature_size":12, "hidden_size":100}


In [ ]:
#Define a GNN (Graph Neural Network) class as a subclass of nn.Module
class GNN(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        # Create the first GraphConv layer with input size equal to the node feature size and output size equal to the hidden size.
        self.conv1 = GraphConv(self.node_feature_size, self.hidden_size, allow_zero_in_degree = True)

        # Create the second GraphConv layer with input size equal to the hidden size and output size equal to the number of tasks
        self.conv2 = GraphConv(self.hidden_size, self.num_tasks, allow_zero_in_degree = True)

    # def forward(self, g, in_feat):
    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

#### Function to Compute Score of the Model

In [ ]:
from sklearn.metrics import roc_auc_score

def compute_score(model, data_loader, val_size, num_tasks):
    # Set the model to evaluation mode
    model.eval()

    # Define the metric as roc_auc_score
    metric = roc_auc_score

    with torch.no_grad():
        # Initialize empty tensors for storing predictions, labels, and masks
        prediction_all = torch.empty(0)
        labels_all = torch.empty(0)
        masks_all = torch.empty(0)

        # Iterate over the data_loader
        for i, (mol_dgl_graph, labels, masks, globals) in enumerate(data_loader):
            # Compute predictions from the model
            prediction = model(mol_dgl_graph, globals)
            prediction = torch.sigmoid(prediction)

            # Concatenate predictions, labels, and masks to the respective tensors
            prediction_all = torch.cat((prediction_all, prediction), 0)
            labels_all = torch.cat((labels_all, labels), 0)
            masks_all = torch.cat((masks_all, masks), 0)

        # Initialize average score tensor
        average = torch.tensor([0.])

        # Compute the metric for each task
        for i in range(num_tasks):
            a1 = prediction_all[:, i][masks_all[:, i] == 1]
            a2 = labels_all[:, i][masks_all[:, i] == 1]

            try:
                # Calculate the metric score
                t = metric(a2.int().cpu(), a1.cpu()).item()
            except ValueError:
                # Handle the case where the metric calculation throws a ValueError
                t = 0

            average += t

    # Return the average score divided by the number of tasks
    return average.item() / num_tasks


#### Loss Function

In [ ]:
# Define the loss function
def loss_func(output, label, mask, num_tasks):
    # Create a tensor of ones as positive weights for BCEWithLogitsLoss
    pos_weight = torch.ones((1, num_tasks))
    pos_weight

    # Define the criterion as BCEWithLogitsLoss with no reduction and positive weights
    criterion = torch.nn.BCEWithLogitsLoss(reduction='none', pos_weight=pos_weight)

    # Compute the element-wise loss by applying the mask to the criterion output
    loss = mask * criterion(output, label)

    # Compute the average loss by summing the masked loss values and dividing by the sum of the mask values
    loss = loss.sum() / mask.sum()

    # Return the computed loss
    return loss



#### Training and Evaluation

##### Training Function

In [ ]:
# Define a function to train a single epoch using the given training data loader, model, and optimizer.
def train_epoch(train_dataloader, model, optimizer):
    # Initialize the epoch train loss and iterations.
    epoch_train_loss = 0
    iterations = 0

    # Set the model to train mode.
    model.train()

    # Iterate over the training data loader.
    for i, (mol_dgl_graph, labels, masks, globals) in enumerate(train_dataloader):
        # Make predictions using the model.
        prediction = model(mol_dgl_graph, globals)

        # Compute the training loss using the loss function.
        loss_train = loss_func(prediction, labels, masks, num_tasks)

        # Zero the gradients of the model parameters.
        optimizer.zero_grad(set_to_none=True)

        # Perform backpropagation to compute gradients.
        loss_train.backward()

        # Update the model parameters using the optimizer.
        optimizer.step()

        # Accumulate the training loss.
        epoch_train_loss += loss_train.detach().item()

        # Increment the iterations count.
        iterations += 1

    # Compute the average epoch train loss.
    epoch_train_loss /= iterations

    # Return the average epoch train loss.
    return epoch_train_loss


In [ ]:
# Define a function to train and evaluate the model.
def train_evaluate():
    # Create a new instance of the GNN model with the given configuration.
    model = GNN(config, global_size, num_tasks)

    # Create an Adam optimizer for training the model.
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

    # Initialize variables for tracking the best validation score and patience count.
    best_val = 0
    patience_count = 1
    epoch = 1

    # Continue training until reaching the maximum number of epochs.
    while epoch <= num_epochs:
        # Check if the patience count is within the allowed limit.
        if patience_count <= patience:
            # Set the model to train mode and compute the training loss for the current epoch.
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)

            # Set the model to eval mode and compute the validation score.
            model.eval()
            score_val = compute_score(model, val_dataloader, len(val_set), num_tasks)

            # Check if the current validation score is better than the best validation score so far.
            if score_val > best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')

                # Create a dictionary to store the checkpoint information.
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})

                # Save the checkpoint to a file using cloudpickle.
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)

                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            # Print the training and validation scores for the current epoch.
            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(epoch, num_epochs, loss_train, score_val))
            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")

        epoch += 1

    # Save the best model by copying the checkpoint directory.
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    # Print the final results.
    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")


##### Function to compute test set score of the final saved model

##### Train the model and evaluate its performance

In [ ]:
def test_evaluate():
    # Create the final model
    final_model = GNN(config, global_size, num_tasks)

    # Set the path to the best model checkpoint file
    path = os.path.join(best_model_path, 'checkpoint.pth')

    # Open the best model checkpoint file and load it
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)

    # Load the state dictionary of the best model
    final_model.load_state_dict(checkpoint["model_state_dict"])

    # Set the final model to evaluation mode
    final_model.eval()

    # Compute the test score
    test_score = compute_score(final_model, test_dataloader, len(test_set), num_tasks)

    # Print the test score
    print("Test Score: {:.3f}".format(test_score), "\n")

    # Print the execution time
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))


In [ ]:
#This line imports the time module, which provides various time-related functions
import time
#This line records the current time using time.time() and assigns it to the variable start_time. It serves as the starting point for measuring the execution time
start_time = time.time()
#This line calls the train_evaluate() function, which is likely responsible for training and evaluating a model.
train_evaluate()
#This line calls the test_evaluate() function, which probably performs evaluation on a separate test dataset.
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 0.695 | Valid Score: 0.285
 
Epoch: 1/100 | Best Valid Score Until Now: 0.285 

Patience 1
Epoch: 2/100 | Training Loss: 0.657 | Valid Score: 0.255
 
Epoch: 2/100 | Best Valid Score Until Now: 0.285 

Patience 2
Epoch: 3/100 | Training Loss: 0.633 | Valid Score: 0.255
 
Epoch: 3/100 | Best Valid Score Until Now: 0.285 

Patience 3
Epoch: 4/100 | Training Loss: 0.615 | Valid Score: 0.262
 
Epoch: 4/100 | Best Valid Score Until Now: 0.285 

Patience 4
Epoch: 5/100 | Training Loss: 0.605 | Valid Score: 0.269
 
Epoch: 5/100 | Best Valid Score Until Now: 0.285 

Patience 5
Epoch: 6/100 | Training Loss: 0.599 | Valid Score: 0.278
 
Epoch: 6/100 | Best Valid Score Until Now: 0.285 

Save checkpoint
Epoch: 7/100 | Training Loss: 0.594 | Valid Score: 0.287
 
Epoch: 7/100 | Best Valid Score Until Now: 0.287 

Save checkpoint
Epoch: 8/100 | Training Loss: 0.590 | Valid Score: 0.300
 
Epoch: 8/100 | Best Valid Score Until Now: 0.300 

Save checkpoint
E

#GNN 2

In [29]:
class GNN(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = SAGEConv(self.node_feature_size, self.hidden_size, aggregator_type='mean')
        self.conv2 = SAGEConv(self.hidden_size, self.num_tasks, aggregator_type='mean')

    # def forward(self, g, in_feat):
    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

In [30]:
from sklearn.metrics import roc_auc_score

def compute_score(model, data_loader, val_size, num_tasks):
    # A function to compute the performance score of the model on the given data
    # model: the trained model to be evaluated
    # data_loader: the data loader for the validation or test set
    model.eval()
    metric = roc_auc_score
    with torch.no_grad():
        # Initialize empty tensors to store the predictions, labels, and masks
        prediction_all= torch.empty(0)
        labels_all= torch.empty(0)
        masks_all= torch.empty(0)
        # Iterate over the data loader and make predictions for each batch
        for i, (mol_dgl_graph, labels, masks, globals) in enumerate(data_loader):
            prediction = model(mol_dgl_graph, globals)
            prediction = torch.sigmoid(prediction)
            # Concatenate the predictions, labels, and masks for all batches
            prediction_all = torch.cat((prediction_all, prediction), 0)
            labels_all = torch.cat((labels_all, labels), 0)
            masks_all = torch.cat((masks_all, masks), 0)
        # Compute the average score across all tasks
        average = torch.tensor([0.])
        for i in range(num_tasks):
            a1 = prediction_all[:, i][masks_all[:,i]==1]
            a2 = labels_all[:, i][masks_all[:,i]==1]
            try:
                # Compute the metric score for the current task
                t = metric(a2.int().cpu(), a1.cpu()).item()
            except ValueError:
                # If the metric cannot be computed, set the score to 0
                t = 0
            average += t
    return average.item()/num_tasks

In [31]:
def loss_func(output, label, mask, num_tasks):
    # A function to compute the loss between the model's output and the true labels
    # mask: the binary mask indicating which samples should be included in the loss calculation
    pos_weight = torch.ones((1, num_tasks))
    # Create a tensor of ones with the same shape as the output tensor (to be used as positive weights)
    pos_weight
    criterion = torch.nn.BCEWithLogitsLoss(reduction='none', pos_weight=pos_weight)
    # Create a loss function that applies binary cross-entropy with logits
    # The 'none' reduction parameter is used to keep the loss for each sample separate
    # The positive weights are used to give more weight to the positive class (to handle class imbalance)
    loss = mask*criterion(output,label)
    # Compute the loss for each sample, but only include the samples specified by the mask
    loss = loss.sum() / mask.sum()
    return loss

In [32]:
def train_epoch(train_dataloader, model, optimizer):
    # A function to train the model for one epoch on the given data
    epoch_train_loss = 0
    iterations = 0
    model.train() # Prepare model for training
    for i, (mol_dgl_graph, labels, masks, globals) in enumerate(train_dataloader):
        # Iterate over the data loader and get the input samples
        prediction = model(mol_dgl_graph, globals)
        # Make predictions for the input samples using the model
        loss_train = loss_func(prediction, labels, masks, num_tasks)
        # Compute the loss between the predictions and the true labels
        optimizer.zero_grad(set_to_none=True)
        # Zero the gradients of the optimizer (to prevent accumulation from previous iterations)
        loss_train.backward()
        # Compute the gradients of the loss with respect to the model parameters
        optimizer.step()
        # Update the model parameters using the computed gradients
        epoch_train_loss += loss_train.detach().item()
        iterations += 1
    epoch_train_loss /= iterations
    return epoch_train_loss

In [33]:
def train_evaluate():
    # A function to train and evaluate the model
    model = GNN(config, global_size, num_tasks)
    # Create a new instance of the GNN1 model
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
    # Create an Adam optimizer to update the model parameters during training
    best_val = 0
    patience_count = 1
    epoch = 1
    while epoch <= num_epochs:
        # Train and evaluate the model for the specified number of epochs
        if patience_count <= patience:
            # Check if the patience count has been exceeded
            model.train()
            # Set the model to training mode
            loss_train = train_epoch(train_dataloader, model, optimizer)
            # Train the model on the training set for one epoch
            model.eval()
            # Set the model to evaluation mode
            score_val = compute_score(model, val_dataloader, len(val_set), num_tasks)
            # Evaluate the model on the validation set and compute the validation score
            if score_val > best_val:
                # Check if the current validation score is better than the best validation score so far
                best_val = score_val
                # If so, update the best validation score and save a checkpoint of the model
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                # If not, increment the patience count
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))
            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    # Save the best model
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)
    # Remove any existing best model and copy the checkpoint to the best model path
    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")
    # Print the final average validation score

In [34]:
def test_evaluate():
    # A function to evaluate the trained model on the test set
    final_model = GNN(config, global_size, num_tasks)
    # Create a new instance of the GNN1 model
    path = os.path.join(best_model_path, 'checkpoint.pth')
    # Load the best model checkpoint from the best model path
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    # Load the saved model state dictionary into the final model
    final_model.eval()
    # Set the final model to evaluation mode
    test_score = compute_score(final_model, test_dataloader, len(test_set), num_tasks)
    # Evaluate the final model on the test set and compute the test score
    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))

In [35]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 0.588 | Valid Score: 0.669
 
Epoch: 1/100 | Best Valid Score Until Now: 0.669 

Save checkpoint
Epoch: 2/100 | Training Loss: 0.495 | Valid Score: 0.855
 
Epoch: 2/100 | Best Valid Score Until Now: 0.855 

Patience 1
Epoch: 3/100 | Training Loss: 0.463 | Valid Score: 0.848
 
Epoch: 3/100 | Best Valid Score Until Now: 0.855 

Patience 2
Epoch: 4/100 | Training Loss: 0.440 | Valid Score: 0.846
 
Epoch: 4/100 | Best Valid Score Until Now: 0.855 

Patience 3
Epoch: 5/100 | Training Loss: 0.423 | Valid Score: 0.843
 
Epoch: 5/100 | Best Valid Score Until Now: 0.855 

Patience 4
Epoch: 6/100 | Training Loss: 0.414 | Valid Score: 0.841
 
Epoch: 6/100 | Best Valid Score Until Now: 0.855 

Patience 5
Epoch: 7/100 | Training Loss: 0.406 | Valid Score: 0.841
 
Epoch: 7/100 | Best Valid Score Until Now: 0.855 

Patience 6
Epoch: 8/100 | Training Loss: 0.405 | Valid Score: 0.832
 
Epoch: 8/100 | Best Valid Score Until Now: 0.855 

Patience 7
Epoch: 9/10

#GNN3


In [51]:
class GNN(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = GINConv(nn.Linear(self.node_feature_size, self.hidden_size), aggregator_type='sum')
        self.conv2 = GINConv(nn.Linear(self.hidden_size, self.num_tasks), aggregator_type='sum')

    # def forward(self, g, in_feat):
    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

In [52]:
from sklearn.metrics import roc_auc_score

def compute_score(model, data_loader, val_size, num_tasks):
    # A function to compute the performance score of the model on the given data
    # model: the trained model to be evaluated
    # data_loader: the data loader for the validation or test set
    model.eval()
    metric = roc_auc_score
    with torch.no_grad():
        # Initialize empty tensors to store the predictions, labels, and masks
        prediction_all= torch.empty(0)
        labels_all= torch.empty(0)
        masks_all= torch.empty(0)
        # Iterate over the data loader and make predictions for each batch
        for i, (mol_dgl_graph, labels, masks, globals) in enumerate(data_loader):
            prediction = model(mol_dgl_graph, globals)
            prediction = torch.sigmoid(prediction)
            # Concatenate the predictions, labels, and masks for all batches
            prediction_all = torch.cat((prediction_all, prediction), 0)
            labels_all = torch.cat((labels_all, labels), 0)
            masks_all = torch.cat((masks_all, masks), 0)
        # Compute the average score across all tasks
        average = torch.tensor([0.])
        for i in range(num_tasks):
            a1 = prediction_all[:, i][masks_all[:,i]==1]
            a2 = labels_all[:, i][masks_all[:,i]==1]
            try:
                # Compute the metric score for the current task
                t = metric(a2.int().cpu(), a1.cpu()).item()
            except ValueError:
                # If the metric cannot be computed, set the score to 0
                t = 0
            average += t
    return average.item()/num_tasks

In [53]:
def loss_func(output, label, mask, num_tasks):
    pos_weight = torch.ones((1, num_tasks))
    pos_weight
    criterion = torch.nn.BCEWithLogitsLoss(reduction='none', pos_weight=pos_weight)
    loss = mask*criterion(output,label)
    loss = loss.sum() / mask.sum()
    return loss

In [54]:
def train_epoch(train_dataloader, model, optimizer):
    epoch_train_loss = 0
    iterations = 0
    model.train() # Prepare model for training
    for i, (mol_dgl_graph, labels, masks, globals) in enumerate(train_dataloader):
        prediction = model(mol_dgl_graph, globals)
        loss_train = loss_func(prediction, labels, masks, num_tasks)
        optimizer.zero_grad(set_to_none=True)
        loss_train.backward()
        optimizer.step()
        epoch_train_loss += loss_train.detach().item()
        iterations += 1
    epoch_train_loss /= iterations
    return epoch_train_loss

In [55]:
def train_evaluate():

    model = GNN(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

    best_val = 0
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)
            model.eval()
            score_val = compute_score(model, val_dataloader, len(val_set), num_tasks)
            if score_val > best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    # best model save
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")


In [56]:
def test_evaluate():
    final_model = GNN(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, len(test_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))

In [57]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 0.634 | Valid Score: 0.313
 
Epoch: 1/100 | Best Valid Score Until Now: 0.313 

Save checkpoint
Epoch: 2/100 | Training Loss: 0.612 | Valid Score: 0.351
 
Epoch: 2/100 | Best Valid Score Until Now: 0.351 

Save checkpoint
Epoch: 3/100 | Training Loss: 0.603 | Valid Score: 0.403
 
Epoch: 3/100 | Best Valid Score Until Now: 0.403 

Save checkpoint
Epoch: 4/100 | Training Loss: 0.590 | Valid Score: 0.479
 
Epoch: 4/100 | Best Valid Score Until Now: 0.479 

Save checkpoint
Epoch: 5/100 | Training Loss: 0.574 | Valid Score: 0.542
 
Epoch: 5/100 | Best Valid Score Until Now: 0.542 

Save checkpoint
Epoch: 6/100 | Training Loss: 0.566 | Valid Score: 0.582
 
Epoch: 6/100 | Best Valid Score Until Now: 0.582 

Save checkpoint
Epoch: 7/100 | Training Loss: 0.554 | Valid Score: 0.633
 
Epoch: 7/100 | Best Valid Score Until Now: 0.633 

Save checkpoint
Epoch: 8/100 | Training Loss: 0.548 | Valid Score: 0.657
 
Epoch: 8/100 | Best Valid Score Until Now: 

#GNN4

In [73]:
class GNN(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks
        self.num_heads = 4
        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = GATConv(self.node_feature_size, self.hidden_size, num_heads=self.num_heads, allow_zero_in_degree=True)
        self.conv2 = GATConv(self.hidden_size, self.num_tasks, num_heads=1, allow_zero_in_degree=True)
    # def forward(self, g, in_feat):
    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

In [74]:
import math
from sklearn.metrics import mean_squared_error
def compute_score(model, data_loader, val_size, num_tasks, scaler=None):
  model.eval()
  loss_sum = nn.MSELoss(reduction='sum') # MSE with sum instead of mean, i.e., sum_i[(y_i)^2-(y'_i)^2]
  final_loss = 0
  state = torch.get_rng_state()
  with torch.no_grad():
            for i, (mol_dgl_graph, labels, masks, globals) in enumerate(data_loader):
                prediction = model(mol_dgl_graph, globals)
                if scaler is not None:
                  prediction = torch.tensor(scaler.inverse_transform(prediction.detach().cpu()))
                  labels = torch.tensor(scaler.inverse_transform(labels.cpu()))
                loss = loss_sum(prediction, labels)
                final_loss += loss.item()
            final_loss /= val_size
            final_loss = math.sqrt(final_loss) # RMSE
  return final_loss / num_tasks

In [75]:
def loss_func(output, label, mask, num_tasks):
    pos_weight = torch.ones((1, num_tasks))
    pos_weight
    criterion = nn.MSELoss(reduction='none')
    loss = mask*criterion(output,label)
    loss = loss.sum() / mask.sum()
    return loss

In [76]:
def train_epoch(train_dataloader, model, optimizer):
    epoch_train_loss = 0
    iterations = 0
    model.train() # Prepare model for training
    for i, (mol_dgl_graph, labels, masks, globals) in enumerate(train_dataloader):
        prediction = model(mol_dgl_graph, globals)
        loss_train = loss_func(prediction, labels, masks, num_tasks)
        optimizer.zero_grad(set_to_none=True)
        loss_train.backward()
        optimizer.step()
        epoch_train_loss += loss_train.detach().item()
        iterations += 1
    epoch_train_loss /= iterations
    return epoch_train_loss

In [77]:
def train_evaluate():

    model = GNN(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

    best_val = 0
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)
            model.eval()
            score_val = compute_score(model, val_dataloader, len(val_set), num_tasks)
            if score_val > best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    # best model save
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")


In [78]:
def test_evaluate():
    final_model = GNN(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, len(test_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))

In [79]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([64, 1])) that is different to the input size (torch.Size([64, 4, 1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([31, 1])) that is different to the input size (torch.Size([31, 4, 1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Save checkpoint
Epoch: 1/100 | Training Loss: 241.741 | Valid Score: 13.877
 
Epoch: 1/100 | Best Valid Score Until Now: 13.877 



/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([11, 1])) that is different to the input size (torch.Size([11, 4, 1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Patience 1
Epoch: 2/100 | Training Loss: 150.707 | Valid Score: 11.271
 
Epoch: 2/100 | Best Valid Score Until Now: 13.877 

Patience 2
Epoch: 3/100 | Training Loss: 106.150 | Valid Score: 9.679
 
Epoch: 3/100 | Best Valid Score Until Now: 13.877 

Patience 3
Epoch: 4/100 | Training Loss: 84.804 | Valid Score: 8.714
 
Epoch: 4/100 | Best Valid Score Until Now: 13.877 

Patience 4
Epoch: 5/100 | Training Loss: 73.990 | Valid Score: 8.127
 
Epoch: 5/100 | Best Valid Score Until Now: 13.877 

Patience 5
Epoch: 6/100 | Training Loss: 68.350 | Valid Score: 7.749
 
Epoch: 6/100 | Best Valid Score Until Now: 13.877 

Patience 6
Epoch: 7/100 | Training Loss: 64.503 | Valid Score: 7.523
 
Epoch: 7/100 | Best Valid Score Until Now: 13.877 

Patience 7
Epoch: 8/100 | Training Loss: 62.520 | Valid Score: 7.354
 
Epoch: 8/100 | Best Valid Score Until Now: 13.877 

Patience 8
Epoch: 9/100 | Training Loss: 61.547 | Valid Score: 7.260
 
Epoch: 9/100 | Best Valid Score Until Now: 13.877 

Patience 9
Ep

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([13, 1])) that is different to the input size (torch.Size([13, 4, 1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
